In [199]:
import random
import sys
sys.path.insert(0,'/home/zahradnik/PycharmProjects/master_thesis')

In [200]:
from src.modules import conf, fit, df_columns, evl, preprocess
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import numpy as np
import haversine as hs
from gpx_converter import Converter
from datetime import datetime
import statsmodels.api as sm
warnings.simplefilter("ignore")

In [201]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [202]:
# race_day = datetime.strptime('2022-10-08-10-30', '%Y-%m-%d-%H-%M') # Hradec
race_day = datetime.strptime('2022-12-31-10-00', '%Y-%m-%d-%H-%M') # Hradec

In [203]:
data = fit.load_pcls(
        athlete_name='zahradnik',
        activity_type='running',
        path_to_load=conf["Paths"]["pcl"],
    )

100%|██████████| 386/386 [00:00<00:00, 1779.89it/s]


In [204]:
simillar_race_index = race_index = fit.get_race_index(data, "2022-08-17-16-46")

In [205]:
stadion_df=data[simillar_race_index]

In [206]:
df = Converter(input_file='silvestrak.gpx').gpx_to_dataframe()
df['enhanced_altitude'] = df.altitude
df.drop("altitude",axis=1,inplace=True)

In [207]:
def calc_dist(pos1: tuple,pos2: tuple) -> int:
    return hs.haversine(pos1, pos2, unit=hs.Unit.METERS)

In [208]:
## DISTANCE
df['dist_diff'] = [0] + [calc_dist(
    (df['latitude'].iloc[x], df['longitude'].iloc[x]),
    (df['latitude'].iloc[x+1], df['longitude'].iloc[x+1])) for x in range(len(df)-1)]

df['distance'] = df['dist_diff'].cumsum()

In [209]:
df["temp"], df["wind_speed"], df["wind_direct"], df["rain"] = preprocess.get_meteo(
    df.latitude.iloc[0],
    df.longitude.iloc[0],
    df.enhanced_altitude.iloc[0],
    race_day
)
df.drop(['latitude','longitude'],axis=1,inplace=True)

In [210]:
df["slope_ascent"], df["slope_descent"] = preprocess.calc_ascent_descent(df)

df["slope_steep"] = preprocess.calc_slope_steep(df)

df['cadence'] = round(np.mean(stadion_df.cadence))